In [1]:
import os
import pickle
import shutil
import pandas as pd
import numpy as np

C:\Users\goert\AppData\Local\Temp\ipykernel_21180\619488019.py:4: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [2]:
MAIN_PATH = os.path.dirname(os.getcwd())
DATA_SUBPATH = '/data'
DATASET_PATH = MAIN_PATH + DATA_SUBPATH + '/unzip/WESAD'
UNPROCESSED_PATH = MAIN_PATH + DATA_SUBPATH + '/unprocessed/WESAD'

In [3]:
minfolder = 2
maxfolder = 18
folders = [f'S{i}' for i in range(minfolder, maxfolder)]
print(folders)

['S2', 'S3', 'S4', 'S5', 'S6', 'S7', 'S8', 'S9', 'S10', 'S11', 'S12', 'S13', 'S14', 'S15', 'S16', 'S17']


This process unpacking the zip files can take up to 3 minutes. Depending on the computational power available.

In [4]:
shutil.unpack_archive(MAIN_PATH + DATA_SUBPATH + '/WESAD.zip', MAIN_PATH + DATA_SUBPATH + '/unzip')

In [5]:
data_folders = []
for subfolder in folders:
    data_folders.append(f"{MAIN_PATH}{DATA_SUBPATH}/unzip/WESAD/{subfolder}/{subfolder}_E4_Data.zip")
print(data_folders)

['d:\\Master of Applied IT/data/unzip/WESAD/S2/S2_E4_Data.zip', 'd:\\Master of Applied IT/data/unzip/WESAD/S3/S3_E4_Data.zip', 'd:\\Master of Applied IT/data/unzip/WESAD/S4/S4_E4_Data.zip', 'd:\\Master of Applied IT/data/unzip/WESAD/S5/S5_E4_Data.zip', 'd:\\Master of Applied IT/data/unzip/WESAD/S6/S6_E4_Data.zip', 'd:\\Master of Applied IT/data/unzip/WESAD/S7/S7_E4_Data.zip', 'd:\\Master of Applied IT/data/unzip/WESAD/S8/S8_E4_Data.zip', 'd:\\Master of Applied IT/data/unzip/WESAD/S9/S9_E4_Data.zip', 'd:\\Master of Applied IT/data/unzip/WESAD/S10/S10_E4_Data.zip', 'd:\\Master of Applied IT/data/unzip/WESAD/S11/S11_E4_Data.zip', 'd:\\Master of Applied IT/data/unzip/WESAD/S12/S12_E4_Data.zip', 'd:\\Master of Applied IT/data/unzip/WESAD/S13/S13_E4_Data.zip', 'd:\\Master of Applied IT/data/unzip/WESAD/S14/S14_E4_Data.zip', 'd:\\Master of Applied IT/data/unzip/WESAD/S15/S15_E4_Data.zip', 'd:\\Master of Applied IT/data/unzip/WESAD/S16/S16_E4_Data.zip', 'd:\\Master of Applied IT/data/unzip/WES

In [6]:
for folder, data_folder in zip(folders, data_folders):
    try:
        shutil.unpack_archive(data_folder, os.path.join(UNPROCESSED_PATH, f'{folder}'))
        print("Preparing folder:", folder)
    except shutil.ReadError:
        print(f"Skipping {data_folder} as it is not a zip file or missing.")

print("Finished unzipping and preparing files.")

Preparing folder: S2
Preparing folder: S3
Preparing folder: S4
Preparing folder: S5
Preparing folder: S6
Preparing folder: S7
Preparing folder: S8
Preparing folder: S9
Preparing folder: S10
Preparing folder: S11
Skipping d:\Master of Applied IT/data/unzip/WESAD/S12/S12_E4_Data.zip as it is not a zip file or missing.
Preparing folder: S13
Preparing folder: S14
Preparing folder: S15
Preparing folder: S16
Preparing folder: S17
Finished unzipping and preparing files.


In [7]:
from scipy.signal import resample

def resample_signal(signal, source_rate, target_rate):
    # Calculate resampling ratio
    resampling_ratio = target_rate / source_rate
    
    # Calculate new length
    new_length = int(len(signal) * resampling_ratio)
    
    # Resample signal
    resampled_signal = resample(signal, new_length)

    return resampled_signal

In [10]:
import os
import pickle
import numpy as np
import pandas as pd
import scipy.signal

# Define the source and destination paths
source_folder = DATASET_PATH
destination_folder = UNPROCESSED_PATH
target_rate = 4

def create_destination_folder(destination_folder, folder):
    # Create the destination folder if it doesn't exist
    destination_path = os.path.join(destination_folder, folder)
    os.makedirs(destination_path, exist_ok=True)
    return destination_path


# Iterate over the folders
for folder in folders:
    destination_path = create_destination_folder(destination_folder, folder)
        
    # Open the pickle file
    source_path = os.path.join(source_folder, folder, f'{folder}.pkl')
    if not os.path.exists(source_path):
        print(f"Skipping {source_path} as it does not exist.")
        continue
        
    with open(source_path, 'rb') as file:
        data = pickle.load(file, encoding='latin1')
        
         # Resample signals
        c_ax_resampled = scipy.signal.resample(data['signal']['chest']['ACC'][0:, 0],
                                               int(len(data['signal']['chest']['ACC']) * target_rate / 700))
        c_ay=scipy.signal.resample(data['signal']['chest']['ACC'][0:,1],
                                               int(len(data['signal']['chest']['ACC']) * target_rate / 700))
        c_az=scipy.signal.resample(data['signal']['chest']['ACC'][0:,2],
                                               int(len(data['signal']['chest']['ACC']) * target_rate / 700))
        c_ecg=data['signal']['chest']['ECG'][:,0]#[baseline_to_plot]
        c_emg=data['signal']['chest']['EMG'][:,0]#[baseline_to_plot]
        c_eda=data['signal']['chest']['EDA'][:,0]#[baseline_to_plot]
        c_temp=data['signal']['chest']['Temp'][:,0]#[baseline_to_plot]
        c_resp=data['signal']['chest']['Resp'][:,0]#[baseline_to_plot]
        w_ax=data['signal']['wrist']['ACC'][0:,0]#[stress_to_plot]
        w_ay=data['signal']['wrist']['ACC'][0:,1]#[stress_to_plot]
        w_az=data['signal']['wrist']['ACC'][0:,2]#[stress_to_plot]
        w_bvp=data['signal']['wrist']['BVP'][:,0]#[stress_to_plot]
        w_eda=data['signal']['wrist']['EDA'][:,0]#[stress_to_plot]
        w_temp=data['signal']['wrist']['TEMP'][:,0]#[stress_to_plot]
        w_label=scipy.signal.resample(data['label'],
                                      int(len(data['label'] * target_rate / 700)))

        print("Equated")

        print(len(w_label))
        print(len(c_eda))
        print(len(w_ax))
        print(len(w_eda))
            
        print(f"Equated: {folder}")


Equated
4255300
4255300
194528
24316
Equated: S2
Equated
4545100
4545100
207776
25972
Equated: S3
Equated
4496100
4496100
205536
25692
Equated: S4
Equated
4380600
4380600
200256
25032
Equated: S5
Equated
4949700
4949700
226272
28284
Equated: S6
Equated
3666600
3666600
167616
20952
Equated: S7
Equated
3826200
3826200
174912
21864
Equated: S8
Equated
3656100
3656100
167136
20892
Equated: S9
Equated
3847200
3847200
175872
21984
Equated: S10
Equated
3663100
3663100
167456
20932
Equated: S11
Skipping d:\Master of Applied IT/data/unzip/WESAD\S12\S12.pkl as it does not exist.
Equated
3875900
3875900
177184
22148
Equated: S13
Equated
3883600
3883600
177536
22192
Equated: S14
Equated
3676400
3676400
168064
21008
Equated: S15
Equated
3941700
3941700
180192
22524
Equated: S16
Equated
4144000
4144000
189440
23680
Equated: S17
